In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import signal

#Epoching

In [ ]:
X=[]
Y=[]
butter_order = 8 # Order of the filter
f_cut_low = 0.1 # Low cut frequency [hz]
f_cut_high = 30 # High cut frequency [hz]
band_type = 'bandpass'
f_s = 250

sos = signal.butter(butter_order, [f_cut_low, f_cut_high], 
                    band_type, fs= f_s, output='sos', analog=False)
for i in range(1,4):
  temp = loadmat("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/RAW Competition Data/S"+str(i)+".mat")
  X_temp = temp['y']
  Y_temp = temp['trig']
  idx = np.where(Y_temp!=0)
  print(idx[0].shape)
  for j in range (0,idx[0].shape[0]):
    epoch = X_temp[idx[0][j]:idx[0][j]+200,:]
    epoch = signal.sosfiltfilt(sos, epoch,axis=0) # Filtering of the epoch
    X.append(epoch)
    Y.append(Y_temp[idx[0][j]])
X = np.array(X)
Y= np.reshape(np.array(Y),(np.array(Y).shape[0]))
X.shape, Y.shape

(1200,)
(1200,)
(1200,)


((3600, 200, 8), (3600,))

In [ ]:
X_test=[]
Y_test=[]
for i in range(4,6):
  temp = loadmat("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/RAW Competition Data/S"+str(i)+".mat")
  X_temp = temp['y']
  Y_temp = temp['trig']
  idx = np.where(Y_temp!=0)
  print(idx[0].shape)
  for j in range (0,idx[0].shape[0]):
    epoch = X_temp[idx[0][j]:idx[0][j]+200,:]
    epoch = signal.sosfiltfilt(sos, epoch,axis=0) # Filtering of the epoch
    X_test.append(epoch)
    Y_test.append(Y_temp[idx[0][j]])
X_test = np.array(X_test)
Y_test= np.reshape(np.array(Y_test),(np.array(Y_test).shape[0]))
X_test.shape, Y_test.shape

(1200,)
(1200,)


((2400, 200, 8), (2400,))

In [ ]:
idx_temp1 =np.where(Y==1)
idx_temp2 =np.where(Y==-1)
idx_temp1[0].shape,idx_temp2[0].shape

((450,), (3150,))

RandomOverSampling

In [ ]:
!pip install imbalanced-learn
import imblearn
print(imblearn.__version__)
from imblearn.over_sampling import RandomOverSampler

0.4.3


In [ ]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy=0.8)
X.shape

(3600, 200, 8)

Converting it to a 2 dim array

In [ ]:
x,y,z=X.shape
X=X.reshape(x,y*z)
x,y,z=X_test.shape
X_test=X_test.reshape(x,y*z)

In [ ]:
# fit and apply the transform
X_over, Y_over = oversample.fit_resample(X, Y)
X_test, Y_test = oversample.fit_resample(X_test, Y_test)
X_over.shape, Y_over.shape

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


((5670, 1600), (5670,))

In [ ]:
idx_temp1 =np.where(Y_over==1)
idx_temp2 =np.where(Y_over==-1)
idx_temp3 =np.where(Y_test ==1)
idx_temp4 =np.where(Y_test==-1)
idx_temp1[0].shape,idx_temp2[0].shape,idx_temp3[0].shape,idx_temp4[0].shape

((2520,), (3150,), (1680,), (2100,))

#Reshaping to apply XDAWN

In [ ]:
X=X_over.reshape(X_over.shape[0],y,z)
X_test =X_test.reshape(X_test.shape[0],y,z)

In [ ]:
X.shape,X_test.shape,Y_test.shape

((5670, 200, 8), (3780, 200, 8), (3780,))

In [ ]:
Y=Y_over

In [ ]:
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/Epoched Data/X_train.npy",X)
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/Epoched Data/X_test.npy",X_test)
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/Epoched Data/Y_train.npy",Y)
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/Epoched Data/Y_test.npy",Y_test)

#Xdawn COvariances

In [ ]:
pip install pyriemann

In [ ]:
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace

In [ ]:
# train_data
XC = XdawnCovariances(nfilter=10,estimator='oas',xdawn_estimator='oas') # our transformer
TS = TangentSpace(metric='riemann')
# collapse first two dimension
train_data = np.swapaxes(X,1,2)
test_data = np.swapaxes(X_test,1,2)
print(train_data.shape,test_data.shape,Y.shape)
# transform our data
X_train = XC.fit_transform(train_data, Y)
X_train = TS.fit_transform(X_train)
X_test = XC.transform(test_data)
X_test = TS.transform(X_test)


(5670, 8, 200) (3780, 8, 200) (5670,)


In [ ]:
print('Transformed training data shape: ' + str(X_train.shape))
print('Training label shape: ' + str(Y.shape))
print('Transformed testing data shape: ' + str(X_test.shape))
print('Testing label shape: ' + str(Y_test.shape))


Transformed training data shape: (5670, 528)
Training label shape: (5670,)
Transformed testing data shape: (3780, 528)
Testing label shape: (3780,)


In [ ]:
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/XDawn/X.npy",X_train)
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/XDawn/Y.npy",Y)
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/XDawn/X_test.npy",X_test)
np.save("/content/drive/MyDrive/2021_VIIT_P300_Session2 /Hackathon Oct 2021/XDawn/Y_test.npy",Y_test)